# 02 - Embedding Generation and Clustering

This notebook generates SPECTER2 embeddings and performs clustering to discover research streams.

**Input:** Parsed papers from notebook 01  
**Output:** Research clusters with embeddings and labels

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

from embeddings import embed_texts, ScientificEmbedder
from clustering import knn_graph, leiden_cluster, hdbscan_cluster, umap_reduce, evaluate_clustering

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries loaded successfully!")

## Load Processed Data

In [ ]:
# Load the analysis-ready dataset
data_dir = Path('../data')
df = pd.read_csv(data_dir / 'parsed_papers_analysis.csv')

print(f"Loaded {len(df)} papers for analysis")
print(f"Columns: {list(df.columns)}")

# Check text availability
print(f"\nText statistics:")
print(f"Papers with text: {df['text'].notna().sum()}")
print(f"Mean text length: {df['text'].str.len().mean():.0f} characters")
print(f"Papers with abstracts: {df['has_abstract'].sum() if 'has_abstract' in df.columns else 'N/A'}")

## Generate SPECTER2 Embeddings

In [ ]:
# Check if embeddings already exist
embeddings_path = data_dir / 'embeddings_specter2.npy'

if embeddings_path.exists():
    print("Loading existing embeddings...")
    embeddings = np.load(embeddings_path)
    print(f"Loaded embeddings shape: {embeddings.shape}")
else:
    print("Generating SPECTER2 embeddings...")
    print("This may take several minutes depending on dataset size and hardware.")
    
    # Initialize embedder
    embedder = ScientificEmbedder(model_name="allenai/specter2")
    
    # Generate embeddings
    texts = df['text'].fillna('').tolist()
    embeddings = embedder.embed_texts(
        texts, 
        batch_size=16,  # Adjust based on GPU memory
        show_progress=True,
        normalize=True
    )
    
    # Save embeddings
    embedder.save_embeddings(embeddings, embeddings_path)
    print(f"Saved embeddings to {embeddings_path}")

print(f"\nEmbedding statistics:")
print(f"Shape: {embeddings.shape}")
print(f"Mean norm: {np.linalg.norm(embeddings, axis=1).mean():.3f}")
print(f"Std norm: {np.linalg.norm(embeddings, axis=1).std():.3f}")

## Dimensionality Reduction for Visualization

In [ ]:
# Generate UMAP embedding for visualization
umap_path = data_dir / 'embeddings_umap2d.npy'

if umap_path.exists():
    print("Loading existing UMAP embeddings...")
    umap_2d = np.load(umap_path)
else:
    print("Generating UMAP 2D embeddings for visualization...")
    umap_2d = umap_reduce(
        embeddings,
        n_components=2,
        n_neighbors=15,
        min_dist=0.1,
        metric='cosine',
        random_state=42
    )
    np.save(umap_path, umap_2d)
    print(f"Saved UMAP embeddings to {umap_path}")

print(f"UMAP embedding shape: {umap_2d.shape}")

# Plot UMAP embedding
plt.figure(figsize=(12, 8))
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], alpha=0.6, s=20)
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.title('UMAP Visualization of Paper Embeddings')
plt.grid(True, alpha=0.3)
plt.show()

## Clustering with Leiden Algorithm

In [ ]:
# Build k-NN graph
print("Building k-NN graph...")
k = 15  # Number of nearest neighbors
knn_g = knn_graph(embeddings, k=k, metric='cosine')

print(f"k-NN graph: {knn_g.vcount()} nodes, {knn_g.ecount()} edges")
print(f"Average degree: {2 * knn_g.ecount() / knn_g.vcount():.1f}")

In [ ]:
# Try different resolution parameters for Leiden clustering
resolutions = [0.5, 1.0, 1.5, 2.0]
leiden_results = {}

for resolution in resolutions:
    print(f"\nTesting Leiden with resolution = {resolution}")
    labels = leiden_cluster(knn_g, resolution=resolution, seed=42)
    
    n_clusters = len(set(labels))
    cluster_sizes = pd.Series(labels).value_counts().sort_values(ascending=False)
    
    # Evaluate clustering
    metrics = evaluate_clustering(embeddings, labels, metric='cosine')
    
    leiden_results[resolution] = {
        'labels': labels,
        'n_clusters': n_clusters,
        'largest_cluster': cluster_sizes.iloc[0],
        'smallest_cluster': cluster_sizes.iloc[-1],
        'silhouette': metrics.get('silhouette'),
        'metrics': metrics
    }
    
    print(f"  Clusters: {n_clusters}")
    print(f"  Largest cluster: {cluster_sizes.iloc[0]} papers")
    print(f"  Smallest cluster: {cluster_sizes.iloc[-1]} papers")
    print(f"  Silhouette score: {metrics.get('silhouette', 'N/A')}")

In [ ]:
# Select best resolution based on silhouette score and cluster size distribution
best_resolution = None
best_score = -1

for resolution, results in leiden_results.items():
    silhouette = results.get('silhouette')
    n_clusters = results['n_clusters']
    
    # Prefer solutions with reasonable number of clusters and good silhouette
    if silhouette is not None and 5 <= n_clusters <= 50:
        # Composite score: silhouette + cluster number penalty
        score = silhouette - abs(n_clusters - 15) * 0.01  # Prefer ~15 clusters
        
        if score > best_score:
            best_score = score
            best_resolution = resolution

if best_resolution is None:
    best_resolution = 1.0  # Default fallback

print(f"\nSelected resolution: {best_resolution}")
leiden_labels = leiden_results[best_resolution]['labels']
n_clusters = leiden_results[best_resolution]['n_clusters']

print(f"Final clustering: {n_clusters} clusters")

# Add cluster labels to dataframe
df['cluster'] = leiden_labels

## Alternative Clustering with HDBSCAN

In [ ]:
# Try HDBSCAN for comparison
print("Running HDBSCAN clustering...")

try:
    hdbscan_labels = hdbscan_cluster(
        embeddings,
        min_cluster_size=15,
        min_samples=5,
        metric='cosine'
    )
    
    # Evaluate HDBSCAN
    hdbscan_metrics = evaluate_clustering(embeddings, hdbscan_labels, metric='cosine')
    
    print(f"HDBSCAN results:")
    print(f"  Clusters: {hdbscan_metrics['n_clusters']}")
    print(f"  Noise points: {hdbscan_metrics['n_noise']} ({hdbscan_metrics['noise_ratio']*100:.1f}%)")
    print(f"  Silhouette: {hdbscan_metrics.get('silhouette', 'N/A')}")
    
    df['cluster_hdbscan'] = hdbscan_labels
    
except ImportError:
    print("HDBSCAN not available - skipping")
    hdbscan_labels = None

## Cluster Visualization

In [ ]:
# Visualize Leiden clusters
plt.figure(figsize=(15, 6))

# Leiden clustering
plt.subplot(1, 2, 1)
scatter = plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=leiden_labels, cmap='tab20', alpha=0.7, s=20)
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.title(f'Leiden Clustering ({n_clusters} clusters)')
plt.colorbar(scatter, label='Cluster ID')
plt.grid(True, alpha=0.3)

# HDBSCAN clustering (if available)
if hdbscan_labels is not None:
    plt.subplot(1, 2, 2)
    scatter = plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=hdbscan_labels, cmap='tab20', alpha=0.7, s=20)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.title(f'HDBSCAN Clustering ({hdbscan_metrics["n_clusters"]} clusters)')
    plt.colorbar(scatter, label='Cluster ID')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Cluster Analysis

In [ ]:
# Analyze cluster characteristics
cluster_stats = []

for cluster_id in sorted(df['cluster'].unique()):
    cluster_papers = df[df['cluster'] == cluster_id]
    
    stats = {
        'cluster_id': cluster_id,
        'n_papers': len(cluster_papers),
        'year_min': cluster_papers['year'].min() if cluster_papers['year'].notna().any() else None,
        'year_max': cluster_papers['year'].max() if cluster_papers['year'].notna().any() else None,
        'year_span': cluster_papers['year'].max() - cluster_papers['year'].min() if cluster_papers['year'].notna().any() else None,
        'top_journal': cluster_papers['journal'].mode().iloc[0] if len(cluster_papers['journal'].mode()) > 0 else 'N/A',
        'journal_diversity': cluster_papers['journal'].nunique(),
        'mean_word_count': cluster_papers['word_count'].mean() if 'word_count' in cluster_papers.columns else None
    }
    
    cluster_stats.append(stats)

cluster_df = pd.DataFrame(cluster_stats).sort_values('n_papers', ascending=False)

print("=== Cluster Statistics ===")
display(cluster_df.head(10))

In [ ]:
# Cluster size distribution
cluster_sizes = df['cluster'].value_counts().sort_values(ascending=False)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.bar(range(len(cluster_sizes)), cluster_sizes.values, alpha=0.7)
plt.xlabel('Cluster Rank')
plt.ylabel('Number of Papers')
plt.title('Cluster Size Distribution')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(cluster_sizes.values, bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('Cluster Size')
plt.ylabel('Frequency')
plt.title('Histogram of Cluster Sizes')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Largest cluster: {cluster_sizes.iloc[0]} papers")
print(f"Smallest cluster: {cluster_sizes.iloc[-1]} papers")
print(f"Mean cluster size: {cluster_sizes.mean():.1f} papers")
print(f"Median cluster size: {cluster_sizes.median():.1f} papers")

## Temporal Cluster Analysis

In [ ]:
# Cluster prevalence over time
if 'year' in df.columns:
    # Focus on largest clusters for visualization
    top_clusters = cluster_sizes.head(8).index
    
    cluster_year = df[df['cluster'].isin(top_clusters)].groupby(['year', 'cluster']).size().unstack(fill_value=0)
    
    plt.figure(figsize=(14, 8))
    
    # Line plot
    plt.subplot(2, 1, 1)
    for cluster in top_clusters:
        if cluster in cluster_year.columns:
            plt.plot(cluster_year.index, cluster_year[cluster], marker='o', label=f'Cluster {cluster}', linewidth=2)
    
    plt.xlabel('Year')
    plt.ylabel('Number of Papers')
    plt.title('Cluster Prevalence Over Time (Top 8 Clusters)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    
    # Stacked area plot
    plt.subplot(2, 1, 2)
    plt.stackplot(cluster_year.index, *[cluster_year[c] for c in top_clusters], 
                  labels=[f'Cluster {c}' for c in top_clusters], alpha=0.7)
    plt.xlabel('Year')
    plt.ylabel('Number of Papers')
    plt.title('Cumulative Cluster Prevalence Over Time')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Generate Cluster Labels with BERTopic

In [ ]:
# Optional: Generate human-readable cluster labels using BERTopic
try:
    from bertopic import BERTopic
    
    print("Generating cluster labels with BERTopic...")
    
    # Use our existing embeddings and clusters
    topic_model = BERTopic(
        calculate_probabilities=False,
        verbose=True,
        nr_topics="auto"
    )
    
    # Fit BERTopic (it will create its own topics, but we'll use it for labels)
    texts = df['text'].fillna('').tolist()
    topics, _ = topic_model.fit_transform(texts, embeddings)
    
    # Get topic info
    topic_info = topic_model.get_topic_info()
    print(f"BERTopic found {len(topic_info)} topics")
    
    # Display top topics
    print("\nTop BERTopic topics:")
    display(topic_info.head(10))
    
    # Save BERTopic results
    df['bertopic'] = topics
    
except ImportError:
    print("BERTopic not available - skipping topic labeling")
except Exception as e:
    print(f"BERTopic failed: {e}")

## Save Results

In [ ]:
# Save clustered dataset
df.to_csv(data_dir / 'papers_clustered.csv', index=False)
print(f"Saved clustered dataset: {len(df)} papers with {n_clusters} clusters")

# Save cluster statistics
cluster_df.to_csv(data_dir / 'cluster_stats.csv', index=False)
print("Saved cluster statistics")

# Save UMAP coordinates
umap_df = pd.DataFrame({
    'umap_x': umap_2d[:, 0],
    'umap_y': umap_2d[:, 1],
    'cluster': leiden_labels
})
umap_df.to_csv(data_dir / 'umap_coordinates.csv', index=False)
print("Saved UMAP coordinates")

# Save clustering results summary
clustering_summary = {
    'leiden': {
        'resolution': best_resolution,
        'n_clusters': n_clusters,
        'metrics': leiden_results[best_resolution]['metrics']
    }
}

if hdbscan_labels is not None:
    clustering_summary['hdbscan'] = hdbscan_metrics

import json
with open(data_dir / 'clustering_summary.json', 'w') as f:
    json.dump(clustering_summary, f, indent=2, default=str)
print("Saved clustering summary")

## Summary

This notebook successfully:

1. **Generated SPECTER2 embeddings** for scientific papers using state-of-the-art models
2. **Created UMAP visualizations** to explore the embedding space structure
3. **Applied Leiden clustering** with multiple resolution parameters to find optimal clusters
4. **Compared with HDBSCAN** as an alternative clustering approach
5. **Analyzed cluster characteristics** including size, temporal, and journal distributions
6. **Generated topic labels** using BERTopic for interpretability

**Key Results:**
- Identified **{n_clusters} research clusters** using Leiden algorithm
- Largest cluster contains **{cluster_sizes.iloc[0]} papers**
- Clusters show distinct temporal and thematic patterns
- High-quality embeddings enable downstream network analysis

**Next Steps:**
- Proceed to `03_openalex_enrichment.ipynb` for citation data enrichment
- Use clusters for temporal burst analysis and network construction
- Generate detailed dialog cards for each research stream